# Статическая и динамическая типизация

## Раннее и позднее связывание

**Связывание** -- соответствие между переменными / функциями и областью памяти, в которой хранятся данные.

Раннее связывание -- размещение переменных и функций в памяти планируется до выполнения программы. Значение всех имен в программе должно быть определено до выполнения.

Примеры: большинство статически компилируемых языков программирования -- Си, C++, Fortran, Rust, ...

Позднее связывание -- размещение переменных и функций определяется во время выполнения, поиск значения символов также происходит на этапе выполнения. Также к позднему связыванию относят использование ссылок на функции в Си и виртуальных методов в C++.

Примеры: почти все языки с динамической типизацией.

Раннее связывание в Си:

```c

int NEG_FLAG = 0;

// корректный код
int check(int x)
{
    if (x > 0) 
    {
        return 1;
    }
    else
    {
        return NEG_FLAG;
    }
}

// ошибка компиляции - POS_FLAG определена позже функции

int heck(int x)
{
    if (x > 0) 
    {
        return POS_FLAG;
    }
    else
    {
        return 0;
    }
}

int POS_FLAG = 1;
```

Позднее связывание в Julia:

In [1]:
function check(x)
    if x > 0
        return true
    else
        return FALSY
    end
end

check (generic function with 1 method)

Вызов функции с неопределенным символом не только не вызывает ошибку, но и дает верный результат, если мы не нарываемся на необходимость разрешения неизвестного символа.

In [2]:
check(25)

true

In [3]:
FALSY = false

false

In [4]:
check(-25)

false

### Преимущества разных видов связывания

Раннее связывание -- заранее известны размещение данных в памяти, размер данных, способ работы с ними. Поскольку производимые операции известны на этапе компиляции, компилятор может применить оптимизации - векторизацию, исключение повторных вычислений и т.д. В случае позднего связывания, например, нельзя автоматически исключить повторные вычисления, т.к. до этапа выполнения программы неизвестно, какие именно вычисления будут проводиться.

Позднее связывание -- гибкость, возможность обобщенного программирования.

Пример гибкости использования семантического позднего связывания в Julia:

In [5]:
using LinearAlgebra: norm

function newton_solver(func, df, x0, ftol)
    x = x0
    f = func(x)
    while norm(f) > ftol
        x -= df(x) \ f
        f = func(x)
    end
    return x
end

newton_solver (generic function with 1 method)

Решение скалярного уравнения

In [6]:
newton_solver(x -> x - cos(x), x -> 1 + sin(x), 0.0, 1e-5)

0.739085133385284

Решение системы уравнений

In [7]:
newton_solver(
    x -> [x[1]^2 + x[2]^2 - 4, 4*x[1]^2 - x[2]^2 - 4],
    x -> [
        2*x[1]  2*x[2]
        8*x[1] -2*x[2]
    ],
    [1.0, 1.0],
    1e-5
)

2-element Vector{Float64}:
 1.2649110640673549
 1.549193338549693

В данном случае программист заранее не заботится о том, как именно представлены в памяти x, f, df, что именно означают `norm` и `\` для введённых типов и т.д. При этом функция в итоге работает как для скалярных уравнений, так и для систем, если корректно задано начальное приближение, функция и производная или якобиан.

## Динамическая и статическая типизация

Особенности статической типизации:
1. Типы переменных указываются при объявлении
2. Результат выражения может быть типизирован до непосредственного вычисления.

Особенности динамической типизации:
1. Типы переменных определяются на этапе выполнения программы, переменная может менять свой тип
2. Тип выражения не определен, тип результата выводится на этапе выполнения.

### Связывание и типизация в Julia

Во множестве случаев чисто динамическая типизация приводит к неоправданно большому числу динамических проверок типа. Поэтому в Julia, несмотря на семантически динамическую типизацию и позднее связывание, реализована концепция "как можно более раннего связывания". Это значит, что если транслятор может по типам аргументов функции вывести типы промежуточных переменных и/или результата -- он это делает и компилирует метод для функции статически, близко к тому как это делается в C++.

К примеру, при использовании `newton_solver` со скалярной функцией, компилятор достаточно умён, чтобы определить, что `f = func(x)` всегда будет давать значение `Float64` и т.п.

Функция `code_typed` показывает типы, которые Julia выводит для аргументов функции и промежуточных значений. Макрос `@code_typed f(x, y, ...)` автоматически подставляет типы всех аргументов в вызове. Посмотрим результат для `newton_solver` на данных, тип которых известен:

In [8]:
@code_typed newton_solver(x -> x - cos(x), x -> 1 + sin(x), 0.0, 1e-5)

CodeInfo(
1 ─ %1  = invoke Main.cos(x0::Float64)::Float64
└── %2  = Base.sub_float(x0, %1)::Float64
2 ┄ %3  = φ (#1 => %2, #3 => %13)::Float64
│   %4  = φ (#1 => x0, #3 => %11)::Float64
│   %5  = Base.abs_float(%3)::Float64
│   %6  = Base.lt_float(ftol, %5)::Bool
└──       goto #4 if not %6
3 ─ %8  = invoke Main.sin(%4::Float64)::Float64
│   %9  = Base.add_float(1.0, %8)::Float64
│   %10 = Base.div_float(%3, %9)::Float64
│   %11 = Base.sub_float(%4, %10)::Float64
│   %12 = invoke Main.cos(%11::Float64)::Float64
│   %13 = Base.sub_float(%11, %12)::Float64
└──       goto #2
4 ─       return %4
) => Float64

И в случае, когда все аргументы динамически типизированы (тип задан как `Any`):

In [9]:
code_typed(newton_solver, (Any, Any, Any, Any))

1-element Vector{Any}:
 CodeInfo(
1 ─ %1  = (func)(x0)::Any
2 ┄ %2  = φ (#1 => %1, #6 => %21)::Any
│   %3  = φ (#1 => x0, #6 => %20)::Any
│   %4  = Main.norm(%2)::Any
│   %5  = Main.:>::typeof(>)
│   %6  = (isa)(%4, BigFloat)::Bool
│   %7  = (isa)(ftol, BigFloat)::Bool
│   %8  = (Core.Intrinsics.and_int)(%6, %7)::Bool
└──       goto #4 if not %8
3 ─ %10 = π (%4, BigFloat)
│   %11 = π (ftol, BigFloat)
│   %12 = invoke %5(%10::BigFloat, %11::BigFloat)::Any
└──       goto #5
4 ─ %14 = (%4 > ftol)::Any
└──       goto #5
5 ┄ %16 = φ (#3 => %12, #4 => %14)::Any
└──       goto #7 if not %16
6 ─ %18 = (df)(%3)::Any
│   %19 = (%18 \ %2)::Any
│   %20 = (%3 - %19)::Any
│   %21 = (func)(%20)::Any
└──       goto #2
7 ─       return %3
) => Any

С помощью функции `code_native` и соответствующего макроса можно также посмотреть сгенерированный машинный код

In [10]:
@code_native newton_solver(x -> x - cos(x), x -> 1 + sin(x), 0.0, 1e-5)

	.text
; ┌ @ In[5]:3 within `newton_solver`
	pushq	%r14
	pushq	%rbx
	subq	$72, %rsp
	vmovsd	%xmm1, 16(%rsp)
	vmovsd	%xmm0, 8(%rsp)
; │ @ In[5]:5 within `newton_solver`
; │┌ @ In[10]:1 within `#15`
	movabsq	$cos, %r14
	callq	*%r14
	vmovsd	8(%rsp), %xmm1                  # xmm1 = mem[0],zero
; ││┌ @ float.jl:402 within `-`
	vsubsd	%xmm0, %xmm1, %xmm2
	movabsq	$.rodata.cst16, %rax
; │└└
; │ @ In[5]:6 within `newton_solver`
; │┌ @ generic.jl:668 within `norm` @ generic.jl:668
; ││┌ @ float.jl:524 within `abs`
	vandpd	(%rax), %xmm2, %xmm0
	vmovsd	16(%rsp), %xmm3                 # xmm3 = mem[0],zero
; │└└
; │┌ @ operators.jl:378 within `>`
; ││┌ @ float.jl:444 within `<`
	vucomisd	%xmm3, %xmm0
; │└└
	jbe	L195
; │ @ In[5] within `newton_solver`
	movabsq	$sin, %rbx
	movabsq	$.rodata.cst8, %rcx
	vmovsd	(%rcx), %xmm3                   # xmm3 = mem[0],zero
	vmovsd	%xmm3, 24(%rsp)
	vmovapd	(%rax), %xmm3
	vmovapd	%xmm3, 32(%rsp)
	nopl	(%rax)
L112:
	vmovapd	%xmm2, 48(%rsp)
	vmovsd	%xmm1, 8(%rsp)
; │

In [11]:
code_native(newton_solver, (Any, Any, Any, Any))

; WARNING: This code may not match what actually runs.
	.text
; ┌ @ In[5]:3 within `newton_solver`
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%r15
	pushq	%r14
	pushq	%r13
	pushq	%r12
	pushq	%rbx
	andq	$-32, %rsp
	subq	$128, %rsp
	vxorps	%xmm0, %xmm0, %xmm0
	vmovaps	%ymm0, 32(%rsp)
	movq	$0, 64(%rsp)
	movq	%rsi, 96(%rsp)
	movq	%fs:0, %rax
	movq	-8(%rax), %rcx
	movq	$12, 32(%rsp)
	movq	(%rcx), %rax
	movq	%rax, 40(%rsp)
	leaq	32(%rsp), %rax
	movq	%rcx, 72(%rsp)
	movq	%rax, (%rcx)
	movq	(%rsi), %rdi
	movq	8(%rsi), %rax
	movq	%rax, 80(%rsp)
	movq	16(%rsi), %rbx
	movq	24(%rsi), %r12
; │ @ In[5]:5 within `newton_solver`
	movq	%rbx, 16(%rsp)
	movabsq	$jl_apply_generic, %r15
	leaq	16(%rsp), %r13
	movq	%rdi, 88(%rsp)
	movq	%r13, %rsi
	movl	$1, %edx
	vzeroupper
	callq	*%r15
	movq	%rax, %r14
	nopl	(%rax)
; │ @ In[5] within `newton_solver`
L160:
	movq	%r14, 64(%rsp)
	movq	%rbx, 56(%rsp)
; │ @ In[5]:6 within `newton_solver`
	movq	%r14, 16(%rsp)
	movabsq	$jl_system_image_data, %rdi
	movq	%r13, %rsi
	movl	$1

Видно, что если компилятор не может делать предположения о типах, то машинный код занимает больше места даже с точки зрения листинга (больше 500 строчек против 200 для типизированной версии). Также в листинге для версии, которой дана информация о типах, видим выполнение операций на регистрах, в том числе активное использование регистров `xmm` -- регистров для чисел с плавающей точкой. Для нетипизированной версии же в листинге в основном операции с памятью и вызовы нижележащих функций, которые и должны в итоге сделать всю работу.

Таким образом, при выполнении вызова `newton_solver` с конкретными аргументами можно подгрузить вариант программного кода, в котором память под локальные переменные выделена статически.

Замер времени макросом `@benchmark` показывает, что для решения скалярного уравнения, действительно, не происходит динамического выделения памяти:

In [12]:
@benchmark newton_solver(x -> x - cos(x), x -> 1 + sin(x), 0.0, 1e-5)

BenchmarkTools.Trial: 10000 samples with 963 evaluations.
 Range (min … max):  84.449 ns … 197.615 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     87.171 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   92.156 ns ±  13.212 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▅█▄▇▃▁▂▃▂▃▂ ▁▁▁▁ ▁    ▁       ▁                             ▂
  ███████████████████████████▇▇▇███▇▇▇▇▆▅▇▆▅▅▄▅▃▃▄▄▄▄▄▂▄▄▇▃▂█▇ █
  84.4 ns       Histogram: log(frequency) by time       155 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.